In [1]:
# !pip install transformers -q
# !pip install datasets -q
# !pip install evaluate -q

In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"
import evaluate
# from datasets import load_dataset

import pandas as pd

df_train = pd.read_csv('/home/aflah20082/NLP_Project/Data/PreprocessedData/train_preprocessed.csv')
df_test = pd.read_csv('/home/aflah20082/NLP_Project/Data/PreprocessedData/test_preprocessed.csv')
df_val = pd.read_csv('/home/aflah20082/NLP_Project/Data/PreprocessedData/val_preprocessed.csv')

df_train = df_train[['preprocessed_text', 'label']]
df_test = df_test[['preprocessed_text']]
df_val = df_val[['preprocessed_text', 'label']]

In [3]:
df_train = df_train.rename(columns={'preprocessed_text': 'text', 'label': 'label'})
df_test = df_test.rename(columns={'preprocessed_text': 'text'})
df_val = df_val.rename(columns={'preprocessed_text': 'text', 'label': 'label'})

In [4]:
# Change Label to 0 and 1
df_train['label'] = df_train['label'].replace({'NOT': 1, 'OFF': 0})
df_val['label'] = df_val['label'].replace({'NOT': 1, 'OFF': 0})

In [5]:
import datasets
from datasets import Dataset, DatasetDict
tds = Dataset.from_pandas(df_train[['text', 'label']], preserve_index=False)
vds = Dataset.from_pandas(df_val[['text', 'label']], preserve_index=False)
testds = Dataset.from_pandas(df_test[['text']], preserve_index=False)

ds = DatasetDict()

ds['train'] = tds
ds['validation'] = vds
ds['test'] = testds

In [6]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = ds.map(tokenize_function, batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42)

In [8]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("GroNLP/hateBERT", num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [9]:
import numpy as np
metric1 = evaluate.load("accuracy")
metric2 = evaluate.load("precision")
metric3 = evaluate.load("recall")

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
from transformers import TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("GroNLP/hateBERT", num_labels=2) 
training_args = TrainingArguments(output_dir=f"HateBERTSaves", evaluation_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
predictions = trainer.predict(tokenized_datasets["validation"])
preds = np.argmax(predictions.predictions, axis=-1)
accuracy = metric1.compute(predictions=preds, references=predictions.label_ids)
precision = metric2.compute(predictions=preds, references=predictions.label_ids)
recall = metric3.compute(predictions=preds, references=predictions.label_ids)
# Write to Txt File
with open(f"/home/aflah20082/NLP_Project/Models/BertBaseUncased.txt", "w") as f:
    f.write(f"Accuracy: {accuracy}\n")
    f.write(f"Precision: {precision}\n")
    f.write(f"Recall: {recall}\n")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch,Training Loss,Validation Loss,Accuracy
1,0.506400,0.526772,0.778323
2,0.465300,0.520081,0.774924
3,0.399800,0.522262,0.786254


Saving model checkpoint to BertBaseUncasedSaves/checkpoint-500
Configuration saved in BertBaseUncasedSaves/checkpoint-500/config.json
Model weights saved in BertBaseUncasedSaves/checkpoint-500/pytorch_model.bin
Saving model checkpoint to BertBaseUncasedSaves/checkpoint-1000
Configuration saved in BertBaseUncasedSaves/checkpoint-1000/config.json
Model weights saved in BertBaseUncasedSaves/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 2648
  Batch size = 8
Saving model checkpoint to BertBaseUncasedSaves/checkpoint-1500
Configuration saved in BertBaseUncasedSaves/checkpoint-1500/config.json
Model weights saved in BertBaseUncasedSaves/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to BertBaseUncasedSaves/checkpoint-2000
Configuration saved in BertBaseUncasedSaves/checkpoint-2000/config.json
Mo

In [15]:
checkpoints = ['500', '1000', '1500', '2000', '2500', '3000', '3500']
ls_results = []
for checkpoint in checkpoints:
    model = AutoModelForSequenceClassification.from_pretrained(f"/home/aflah20082/NLP_Project/Models/HateBERTSaves/checkpoint-{checkpoint}", num_labels=2)
    training_args = TrainingArguments(output_dir=f"/home/aflah20082/NLP_Project/Models/HateBERTSaves/checkpoint-{checkpoint}", evaluation_strategy="epoch")
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=small_train_dataset,
        eval_dataset=small_eval_dataset,
        compute_metrics=compute_metrics,
    )
    print(f"Checkpoint: {checkpoint}")
    # Accuracy
    metric = evaluate.load("accuracy")
    # Predictions for Test Data
    predictions = trainer.predict(tokenized_datasets["validation"])
    preds = np.argmax(predictions.predictions, axis=-1)
    accuracy = metric1.compute(predictions=preds, references=predictions.label_ids)
    precision = metric2.compute(predictions=preds, references=predictions.label_ids)
    recall = metric3.compute(predictions=preds, references=predictions.label_ids)
    # Macro F1 Score SKLearn
    from sklearn.metrics import f1_score
    macro_f1 = f1_score(predictions.label_ids, preds, average='macro')
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"Macro F1: {macro_f1}")
    ls_results.append([checkpoint, accuracy, precision, recall, macro_f1])

loading configuration file /home/aflah20082/NLP_Project/Models/BertBaseUncasedSaves/checkpoint-500/config.json
Model config BertConfig {
  "_name_or_path": "/home/aflah20082/NLP_Project/Models/BertBaseUncasedSaves/checkpoint-500",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /home/aflah20082/NLP_Project/Models/BertBaseUncasedSaves/check

Checkpoint: 500


The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 2648
  Batch size = 8


loading configuration file /home/aflah20082/NLP_Project/Models/BertBaseUncasedSaves/checkpoint-1000/config.json
Model config BertConfig {
  "_name_or_path": "/home/aflah20082/NLP_Project/Models/BertBaseUncasedSaves/checkpoint-1000",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /home/aflah20082/NLP_Project/Models/BertBaseUncasedSaves/che

Accuracy: {'accuracy': 0.774546827794562}
Precision: {'precision': 0.7674199623352166}
Recall: {'recall': 0.9405654933641084}
Macro F1: 0.7151725255003267


...

Accuracy: {'accuracy': 0.7613293051359517}
Precision: {'precision': 0.8481973434535104}
Recall: {'recall': 0.773802654356607}
Macro F1: 0.7452120380954237


All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at /home/aflah20082/NLP_Project/Models/BertBaseUncasedSaves/checkpoint-3500.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Checkpoint: 3500


The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 2648
  Batch size = 8


Accuracy: {'accuracy': 0.7719033232628398}
Precision: {'precision': 0.8461066830165542}
Recall: {'recall': 0.7963069821119446}
Macro F1: 0.7539112217290369


In [16]:
# print results
df_results = pd.DataFrame(ls_results, columns=['checkpoint', 'accuracy', 'precision', 'recall', 'macro_f1'])
df_results 

,checkpoint,accuracy,precision,recall,macro_f1
0,500,{'accuracy': 0.774546827794562},{'precision': 0.7674199623352166},{'recall': 0.9405654933641084},0.715173
1,1000,{'accuracy': 0.7904078549848943},{'precision': 0.7921626984126984},{'recall': 0.9215233698788229},0.746601
2,1500,{'accuracy': 0.7794561933534743},{'precision': 0.7742243436754177},{'recall': 0.9359492210040392},0.724810
3,2000,{'accuracy': 0.7734138972809668},{'precision': 0.8303206997084548},{'recall': 0.8216964800923254},0.750655
4,2500,{'accuracy': 0.7851208459214502},{'precision': 0.7803468208092486},{'recall': 0.9347951529140219},0.733984
5,3000,{'accuracy': 0.7613293051359517},{'precision': 0.8481973434535104},{'recall': 0.773802654356607},0.745212
6,3500,{'accuracy': 0.7719033232628398},{'precision': 0.8461066830165542},{'recall': 0.7963069821119446},0.753911


In [17]:
# load Best checkpoint: 3500 Here
model = AutoModelForSequenceClassification.from_pretrained(f"/home/aflah20082/NLP_Project/Models/HateBERTSaves/checkpoint-3500", num_labels=2)
training_args = TrainingArguments(output_dir=f"/home/aflah20082/NLP_Project/Models/HateBERTSaves/checkpoint-3500", evaluation_strategy="epoch", num_train_epochs=1)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_eval_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
# Finetune on Validation Data for 1 Epoch
trainer.train()
# Get Predictions
predictions = trainer.predict(tokenized_datasets["test"])
preds = np.argmax(predictions.predictions, axis=-1)

loading configuration file /home/aflah20082/NLP_Project/Models/BertBaseUncasedSaves/checkpoint-3500/config.json
Model config BertConfig {
  "_name_or_path": "/home/aflah20082/NLP_Project/Models/BertBaseUncasedSaves/checkpoint-3500",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /home/aflah20082/NLP_Project/Models/BertBaseUncasedSaves/che

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.476056,0.798338


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 2648
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 860
  Batch size = 8


In [18]:
ls = []
for i in range(len(preds)):
    ls.append([i, preds[i]])
df = pd.DataFrame(ls, columns=['id', 'label'])

In [19]:
df

,id,label
0,0,0
1,1,0
2,2,0
3,3,1
4,4,1
...,...,...
855,855,0
856,856,1
857,857,0
858,858,1
